<a href="https://colab.research.google.com/github/rockowtham/eip_class/blob/master/Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/eipdata/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D, SeparableConv2D
import seaborn as sns

In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

In [0]:

sns.countplot(x='emotion',data=df)

In [0]:
sns.countplot(x='gender',data=df)

In [0]:

sns.countplot(x='imagequality',data=df)

In [0]:

sns.countplot(x='age',data=df)

In [0]:

sns.countplot(x='weight',data=df)

In [0]:
sns.countplot(x='carryingbag',data=df)

In [0]:
sns.countplot(x='footwear',data=df)

In [0]:
sns.countplot(x='bodypose',data=df)

In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head(10).T

In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        
        images = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])        
        if self.augmentation is not None:
            images = self.augmentation.flow(images, shuffle=False).next()
        
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        
        return images, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

In [0]:
train_df.head()

In [0]:
# create train and validation data generators
# train_gen = PersonDataGenerator(train_df, batch_size=64)
train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, shuffle=True,
    augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
    )
)
valid_gen = PersonDataGenerator(
    val_df, 
    batch_size=32, shuffle=True,
    augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
    )
)
# valid_gen = PersonDataGenerator(val_df, batch_size=128, shuffle=True)

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

In [0]:
backbone = VGG16(
    weights="imagenet", 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output
neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)
neck = BatchNormalization()(neck)


def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(256, activation="relu")(neck)
    neck = MaxPooling2D(pool_size=(2, 2))
    # neck = BatchNormalization()(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = MaxPooling2D(pool_size=(2, 2))
    # neck = BatchNormalization()(neck)
    neck = Dropout(0.25)(in_layer)
    neck = Dense(64, activation="relu")(neck)
    neck = MaxPooling2D(pool_size=(2, 2))
    # neck = BatchNormalization()(neck)
    neck = Dropout(0.25)(in_layer)
    neck = Dense(32, activation="relu")(neck)

    # neck = Dropout(0.2)(in_layer)
    # neck = Dense(128, activation="relu")(neck)
    # neck = BatchNormalization()(neck)
    # neck = Dropout(0.3)(in_layer)
    # # neck = MaxPooling2D(pool_size=(2, 2))
    # neck = Dense(128, activation="relu")(neck)
    # neck = BatchNormalization()(neck)

    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)
from keras.utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [0]:
# freeze backbone
for layer in backbone.layers:
	layer.trainable = False

In [0]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import os
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1
n = 3
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2
# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 15:
        lr *= 0.5e-3
    elif epoch > 20:
        lr *= 1e-3
    elif epoch > 10:
        lr *= 1e-1
    elif epoch > 25:
        lr *= 0.25e-5
    print('Learning rate: ', lr)
    return lr

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
# opt = Adam(lr=lr_schedule(0))
# model.compile(
#     optimizer=opt,
#     loss="categorical_crossentropy", 
#     # loss_weights=loss_weights, 
#     metrics=["accuracy"]
# )

In [0]:
# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)

In [23]:
 model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=35, 
    epochs=50,
    verbose=1,
    callbacks=callbacks
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
Learning rate:  0.001
360/360 [==============================] - 100s 279ms/step - loss: 9.5956 - gender_output_loss: 0.6554 - image_quality_output_loss: 1.1982 - age_output_loss: 1.8394 - weight_output_loss: 1.3717 - bag_output_loss: 1.2634 - footwear_output_loss: 1.0439 - pose_output_loss: 0.9604 - emotion_output_loss: 1.2632 - gender_output_acc: 0.6685 - image_quality_output_acc: 0.4511 - age_output_acc: 0.2703 - weight_output_acc: 0.4886 - bag_output_acc: 0.4388 - footwear_output_acc: 0.5359 - pose_output_acc: 0.5791 - emotion_output_acc: 0.5286 - val_loss: 7.8581 - val_gender_output_loss: 0.5160 - val_image_quality_output_loss: 1.0281 - val_age_output_loss: 1.5985 - val_weight_output_loss: 1.1117 - val_bag_output_loss: 0.9823 - val_footwear_output_loss: 0.8605 - val_pose_output_loss: 0.7279 - val_emotion_output_loss: 1.0332 - val_gender_output_acc: 0.7465 - val_image_quality_

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


Learning rate:  0.001
360/360 [==============================] - 90s 249ms/step - loss: 7.7993 - gender_output_loss: 0.5252 - image_quality_output_loss: 1.0112 - age_output_loss: 1.5788 - weight_output_loss: 1.1106 - bag_output_loss: 0.9688 - footwear_output_loss: 0.8921 - pose_output_loss: 0.7190 - emotion_output_loss: 0.9935 - gender_output_acc: 0.7469 - image_quality_output_acc: 0.5224 - age_output_acc: 0.3418 - weight_output_acc: 0.5850 - bag_output_acc: 0.5703 - footwear_output_acc: 0.6030 - pose_output_acc: 0.6951 - emotion_output_acc: 0.6806 - val_loss: 7.4205 - val_gender_output_loss: 0.4620 - val_image_quality_output_loss: 1.0040 - val_age_output_loss: 1.5455 - val_weight_output_loss: 1.0487 - val_bag_output_loss: 0.9341 - val_footwear_output_loss: 0.7913 - val_pose_output_loss: 0.6453 - val_emotion_output_loss: 0.9896 - val_gender_output_acc: 0.7758 - val_image_quality_output_acc: 0.5109 - val_age_output_acc: 0.3641 - val_weight_output_acc: 0.6017 - val_bag_output_acc: 0.5734

In [24]:
results = model.evaluate_generator(valid_gen, verbose=1)
dict(zip(model.metrics_names, results))

63/63 [==============================] - 12s 197ms/step


{'age_output_acc': 0.39186507936507936,
 'age_output_loss': 1.4253418180677626,
 'bag_output_acc': 0.6215277777777778,
 'bag_output_loss': 0.8644559979438782,
 'emotion_output_acc': 0.6884920634920635,
 'emotion_output_loss': 0.9454269087503827,
 'footwear_output_acc': 0.6884920634920635,
 'footwear_output_loss': 0.7572003451604692,
 'gender_output_acc': 0.8214285714285714,
 'gender_output_loss': 0.40223101609283024,
 'image_quality_output_acc': 0.5466269841269841,
 'image_quality_output_loss': 0.943423298616258,
 'loss': 6.8500970885867165,
 'pose_output_acc': 0.7862103174603174,
 'pose_output_loss': 0.5439886249720104,
 'weight_output_acc': 0.6304563492063492,
 'weight_output_loss': 0.9680290912824964}